<a href="https://colab.research.google.com/github/drusho/data_analysis/blob/main/Exploring_Hackers_News_Posts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Exploring Hackers News Posts</h1>

<p>This project will compare two different types of posts from Hacker News website which specializes in technology related stories, it's very similar to Reddit. 

Two types of posts that will be explore in this notebook begin with either ***Ask HN*** or ***Show HN***.

Users submit Ask HN posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

This notebook will compare these two types of posts to determine the following:

- *Do Ask HN or Show HN receive more comments on average?*
- *Do posts created at a certain time receive more comments on average?*

**Note** *This data set was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.*

<h3>Import Dataset</h3>

In [ ]:
#Import CVS from github
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/drusho/100daysofcode/main/Exploring_Hacker_News_Posts/hacker_news.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3032k  100 3032k    0     0  17.8M      0 --:--:-- --:--:-- --:--:-- 17.9M


In [ ]:
# Read in the data.
import csv

f = open('hacker_news.csv')
hn = list(csv.reader(f))
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

<h3>Separate Header from Data</h3>

In [ ]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 59658  100 59658    0     0   280k      0 --:--:-- --:--:-- --:--:--  278k


In [ ]:
# Remove the headers.
hn_header = hn[0]
hn = hn[1:]
print(hn_header)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


<h3>Filtering 'Ask HN' and 'Show HN' Posts</h3>

Identify posts that begin with either `Ask HN` or `Show HN` and separate the data into different lists.



In [ ]:
# Identify posts that begin with either `Ask HN` or `Show HN` and separate the data into different lists.


ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1]
    if title.lower().startswith("Ask HN".lower()):
        ask_posts.append(post)
    elif title.lower().startswith("Show HN".lower()):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
        

print(f'Len of whole dataset: {len(hn)}')
print(f'Len of ask_posts: {len(ask_posts)}')
print(f'Len of show_posts: {len(show_posts)}')
print(f'Len of other_posts: {len(other_posts)}')
print(f'\n* Data checking:\nLen of ask_posts + show_posts + other_posts = {len(ask_posts)+len(show_posts)+len(other_posts)}')
        

Len of whole dataset: 20100
Len of ask_posts: 1744
Len of show_posts: 1162
Len of other_posts: 17194

* Data checking:
Len of ask_posts + show_posts + other_posts = 20100


<h3>Compare Number of Comments for 'Ask HN' and 'Show HN'</h3>


In [ ]:
print(f"Header: \n{hn_header}")

# Comments found in column 4

# Sample from row 4 of 'ask_posts' list
print(f"\nSample from row 4 of 'ask_posts' list:\n{ask_posts[4]}")
print(f'\nComment = {ask_posts[4][4]}')

Header: 
['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

Sample from row 4 of 'ask_posts' list:
['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']

Comment = 17


In [ ]:
def count_comments(comlist):
    num_comments = 0
    for row in comlist:
        comments = int(row[4])
        num_comments += comments
    return num_comments

def avg_comments(comlist):
    num_comments = count_comments(comlist)
    return round(num_comments / len(comlist))

print('Total Comments')
print(f'  Ask HN: {count_comments(ask_posts)}')
print(f'  Show HN: {count_comments(show_posts)}')
print(f'  Other Posts: {count_comments(other_posts)}')

print('\nLists Averages')
print(f'  Ask HN: {avg_comments(ask_posts)}')
print(f'  Show HN: {avg_comments(show_posts)}')
print(f'  Other Posts: {avg_comments(other_posts)}')  

Total Comments
  Ask HN: 24483
  Show HN: 11988
  Other Posts: 462055

Lists Averages
  Ask HN: 14
  Show HN: 10
  Other Posts: 27


<h2>Determine the Number of 'Ask HN' Comments Created by Hour</h2>

*Ask HN* had a higher average of comments than *Show HN*

Use *Ask HN* to determine the time of day when the most comments are received 

In [ ]:
print(f'Header: \n{hn_header}')
print(f"\nColumn 7 in header = '{hn_header[6]}' ")

print(f'\nSample from row 3 of data set: \n{hn[3][6]}')

Header: 
['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

Column 7 in header = 'created_at' 

Sample from row 3 of data set: 
6/17/2016 0:01


<h3>Convert column 7 in 'Ask HN' to datetime objects</h3>

In [ ]:
import datetime as dt

result_list = []

for post in ask_posts:
    result_list.append(
        [post[6], int(post[4])]
    )

  
comment_created_hr = {} #Number posts per hour
counts_by_hour = {} #Number of comments per hour
date_format = "%m/%d/%Y %H:%M"

# print(result_list[1])

for row in result_list:
    date = row[0]
    comment_num = int(row[1])
    date = dt.datetime.strptime(date,date_format)
    hour = date.hour
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comment_created_hr[hour] = comment_num
    else: 
        counts_by_hour[hour] += 1
        comment_created_hr[hour] += comment_num


comment_created_hr
counts_by_hour
        
    

{0: 55,
 1: 60,
 2: 58,
 3: 54,
 4: 47,
 5: 46,
 6: 44,
 7: 34,
 8: 48,
 9: 45,
 10: 59,
 11: 58,
 12: 73,
 13: 85,
 14: 107,
 15: 116,
 16: 108,
 17: 100,
 18: 109,
 19: 110,
 20: 80,
 21: 109,
 22: 71,
 23: 68}

<h3>Calculating the Average Number of Comments for Ask HN Posts by Hour</h3>

In [ ]:
avg_by_hour = []


for hour in comment_created_hr:
    avg_by_hour.append([hour,round(comment_created_hr[hour]/counts_by_hour[hour],2)])

avg_by_hour

[[9, 5.58],
 [13, 14.74],
 [10, 13.44],
 [14, 13.23],
 [16, 16.8],
 [23, 7.99],
 [12, 9.41],
 [17, 11.46],
 [15, 38.59],
 [21, 16.01],
 [20, 21.52],
 [2, 23.81],
 [18, 13.2],
 [3, 7.8],
 [5, 10.09],
 [19, 10.8],
 [1, 11.38],
 [22, 6.75],
 [8, 10.25],
 [4, 7.17],
 [0, 8.13],
 [6, 9.02],
 [7, 7.85],
 [11, 11.05]]

<h3>Sorting and Printing Values from a List of Lists</h3>

In [ ]:
#Sort list by hour 

sorted(avg_by_hour,reverse=False)

[[0, 8.13],
 [1, 11.38],
 [2, 23.81],
 [3, 7.8],
 [4, 7.17],
 [5, 10.09],
 [6, 9.02],
 [7, 7.85],
 [8, 10.25],
 [9, 5.58],
 [10, 13.44],
 [11, 11.05],
 [12, 9.41],
 [13, 14.74],
 [14, 13.23],
 [15, 38.59],
 [16, 16.8],
 [17, 11.46],
 [18, 13.2],
 [19, 10.8],
 [20, 21.52],
 [21, 16.01],
 [22, 6.75],
 [23, 7.99]]

In [ ]:
# Sort list by averages

avg_by_hour.sort(key=lambda x:x[1],reverse=True)

avg_by_hour

[[15, 38.59],
 [2, 23.81],
 [20, 21.52],
 [16, 16.8],
 [21, 16.01],
 [13, 14.74],
 [10, 13.44],
 [14, 13.23],
 [18, 13.2],
 [17, 11.46],
 [1, 11.38],
 [11, 11.05],
 [19, 10.8],
 [8, 10.25],
 [5, 10.09],
 [12, 9.41],
 [6, 9.02],
 [0, 8.13],
 [23, 7.99],
 [7, 7.85],
 [3, 7.8],
 [4, 7.17],
 [22, 6.75],
 [9, 5.58]]

In [ ]:
swap_avg_by_hour = []

for row in avg_by_hour:
    hour = row[0]
    avg = row[1]
    swap_avg_by_hour.append([avg,hour])

swap_avg_by_hour = sorted(swap_avg_by_hour,reverse=True)

In [ ]:
print('Top 5 Hours for Ask Posts Comments')

for avg, hr in swap_avg_by_hour[:5]:
    print(
        "• {}: {:.2f} average comments per post".format(
            dt.datetime.strptime(str(hr), "%H").strftime("%H:%M"),avg
        )
    )

Top 5 Hours for Ask Posts Comments
• 15:00: 38.59 average comments per post
• 02:00: 23.81 average comments per post
• 20:00: 21.52 average comments per post
• 16:00: 16.80 average comments per post
• 21:00: 16.01 average comments per post


# Conclusion

Results of analyzing both 'Ask HN' or 'Show HN' showed that 'Ask HN' was more popular when it came to total number of comments and total number of posts.